<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 4

###  Big Query, SQL, Classification

---

### The Data

There are 5 individual tables that have the information, contained in a Google BigQuery database.  The setup info for BigQuery is located on our DSI wiki.  You will have to query with SQL, the dataset in order to complete this project.

The tables, with cooresonding attributes that exist are:

### businesses
- business_id: unique business identifier
- name: name of the business
- review_count: number of reviews per business
- city: city business resides in
- stars: average rating
- categories: categories the business falls into (can be one or multiple)
- latitude
- longitude
- neighborhoods: neighborhoods business belongs to
- variable: "property" of the business (a tag)
- value: True/False for the property

### reviews
- user_id: unique user identifier
- review_id: unique review identifier
- votes.cool: how many thought the review was "cool"
- business_id: unique business id the review is for
- votes.funny: how many thought the review was funny
- stars: rating given
- date: date of review
- votes.useful: how many thought the review was useful
- ... 100 columns of counts of most common 2 word phrases that appear in reviews in this review

### users
- yelping_since: signup date
- compliments.plain: # of compliments "plain"
- review_count: # of reviews:
- compliments.cute: total # of compliments "cute"
- compliments.writer: # of compliments "writer"
- compliments.note: # of compliments "note" (not sure what this is)
- compliments.hot: # of compliments "hot" (?)
- compliments.cool: # of compliments "cool"
- compliments.profile: # of compliments "profile"
- average_stars: average rating
- compliments.more: # of compliments "more"
- elite: years considered "elite"
- name: user's name
- user_id: unique user id
- votes.cool: # of votes "cool"
- compliments.list: # of compliments "list"
- votes.funny: # of compliments "funny"
- compliments.photos: # of compliments "photos"
- compliments.funny: # of compliments "funny"
- votes.useful: # of votes "useful"

### checkins
- business_id: unique business identifier
- variable: day-time identifier of checkins (0-0 is Sunday 0:00 - 1:00am,  for example)
- value: # of checkins at that time

### tips
- user_id: unique user identifier
- business_id: unique business identifier
- likes: likes that the tip has
- date: date of tip
- ... 100 columns of counts of most common 2 word phrases that appear in tips in this tip


The reviews and tips datasets in particular have parsed "NLP" columns with counts of 2-word phrases in that review or tip (a "tip", it seems, is some kind of smaller review).

The user dataset has a lot of columns of counts of different compliments and votes. We're not sure whether the compliments or votes are by the user or for the user.

Full details about this dataset area located here:
https://bigquery.cloud.google.com/dataset/bigquery-dsi-dave:yelp_arizona

---


If you look at the website, or the full data, you'll see I have removed pieces of the data and cut it down quite a bit. This is to simplify it for this project. Specifically, business are limited to be in these cities:

- Phoenix
- Surprise
- Las Vegas
- Waterloo

Apparently there is a city called "Surprise" in Arizona. 

Businesses are also restricted to at least be in one of the following categories, because we thought the mix of them was funny:

- Airports
- Breakfast & Brunch
- Bubble Tea
- Burgers
- Bars
- Bakeries
- Breweries
- Cafes
- Candy Stores
- Comedy Clubs
- Courthouses
- Dance Clubs
- Fast Food
- Museums
- Tattoo
- Vape Shops
- Yoga
    
---

### Project requirements

**You will be performing 4 different sections of analysis, like in the last project.**

Remember that classification targets are categorical and regression targets are continuous variables.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Load your dataset(s) / setup / configure GBQ connection

---

Information about this dataset is located here:


**If you haven't done so, setup a project with the Google developer portal, following the directions here: [Getting Started with BigQuery](https://github.com/ga-students/DSI-SF-4/wiki/Getting-Started-with-BigQuery)**

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

%matplotlib inline

project_id = "bigquery-dsi-andrew-stark"

sql = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.businesses] 
LIMIT 2500
"""

df = pd.read_gbq(sql, project_id=project_id)

Requesting query... ok.
Query running...
Query done.
Processed: 25.2 Mb

Retrieving results...
Got 2500 rows.

Total time taken 2.56 s.
Finished at 2016-12-20 08:42:22.


In [2]:
#df.categories.value_counts()
df.head()

,business_id,name,review_count,city,stars,categories,latitude,longitude,neighborhoods,variable,value
0,--jFTZmywe7StuZ2hEjxyA,Subway,7,Las Vegas,3.5,"['Fast Food', 'Sandwiches', 'Restaurants']",36.118819,-115.182005,[],attributes.Takes Reservations,False
1,--jFTZmywe7StuZ2hEjxyA,Subway,7,Las Vegas,3.5,"['Fast Food', 'Sandwiches', 'Restaurants']",36.118819,-115.182005,[],attributes.Good For.dessert,False
2,--jFTZmywe7StuZ2hEjxyA,Subway,7,Las Vegas,3.5,"['Fast Food', 'Sandwiches', 'Restaurants']",36.118819,-115.182005,[],attributes.Take-out,True
3,--jFTZmywe7StuZ2hEjxyA,Subway,7,Las Vegas,3.5,"['Fast Food', 'Sandwiches', 'Restaurants']",36.118819,-115.182005,[],attributes.Has TV,False
4,--jFTZmywe7StuZ2hEjxyA,Subway,7,Las Vegas,3.5,"['Fast Food', 'Sandwiches', 'Restaurants']",36.118819,-115.182005,[],attributes.Good For.breakfast,True


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Constructing a "profile" for Las Vegas

---

Yelp is interested in building out what they are calling "profiles" for cities. They want you to start with just Las Vegas to see what a prototype of this would look like. Essentially, they want to know what makes Las Vegas distinct from the other four.

Use the data you have to predict Las Vegas from the other variables you have. You should not be predicting the city from any kind of location data or other data perfectly associated with that city (or another city).

You may use any classification algorithm you deem appropriate, or even multiple models. You should:

1. Build at least one model predicting Las Vegas vs. the other cities.
- Validate your model(s).
- Interpret and visualize, in some way, the results.
- Write up a "profile" for Las Vegas. This should be a writeup converting your findings from the model(s) into a human-readable description of the city.

*Research location data to find the city targets.*

In [3]:
#Create a column "is_lasvegas" where 0 is no, 1 is yes
df['is_lasvegas'] = df['city'].map(lambda x: 1 if x == "Las Vegas" else 0)

In [5]:
df.is_lasvegas.value_counts()

0    2141
1     359
Name: is_lasvegas, dtype: int64

In [6]:
# The following are the top 20 most common category combinations for the businesses
# Create 20 columns "is_profile_x" to track if the restaurant has a given category combination

#1 ['Burgers', 'Fast Food', 'Restaurants']                                                          194
#2 ['Breakfast & Brunch', 'American (Traditional)', 'Restaurants']                                  138
#3 ['Wine Bars', 'Bars', 'American (New)', 'Nightlife', 'Restaurants']                              120
#4 ['Fast Food', 'Restaurants']                                                                     117
#5 ['Fast Food', 'Mexican', 'Restaurants']                                                           95
#6 ['American (Traditional)', 'Fast Food', 'Restaurants']                                            89
#7 ['Fast Food', 'Sandwiches', 'Restaurants']                                                        79
#8 ['Burgers', 'Restaurants']                                                                        75
#9 ['Breakfast & Brunch', 'Restaurants']                                                             73
#10 ['Delis', 'Fast Food', 'Sandwiches', 'Restaurants']                                               65
#11 ['Puerto Rican', 'Bars', 'Latin American', 'Caribbean', 'Nightlife', 'Restaurants']               58
#12 ['Irish', 'Music Venues', 'Arts & Entertainment', 'Bars', 'Pubs', 'Nightlife', 'Restaurants']     58
#13 ['Bars', 'Restaurants', 'American (Traditional)', 'Karaoke', 'Nightlife']                         58
#14 ['Bars', 'American (Traditional)', 'Nightlife', 'Restaurants']                                    56
#15 ['Bars', 'Food', 'Beer, Wine & Spirits', 'Dive Bars', 'Nightlife', 'Karaoke']                     56
#16 ['Bars', 'Restaurants', 'American (Traditional)', 'Sports Bars', 'Nightlife']                     55
#17 ['Breakfast & Brunch', 'Cafes', 'American (Traditional)', 'Restaurants']                          55
#18 ['Burgers', 'American (Traditional)', 'Restaurants']                                              52
#19 ['Fast Food', 'Mexican', 'Tex-Mex', 'Restaurants']                                                51
#20 ['Breakfast & Brunch', 'American (Traditional)', 'Diners', 'Restaurants']                         51

df['is_profile_1'] = df['categories'].map(lambda x: 1 if x == "['Burgers', 'Fast Food', 'Restaurants']" else 0)
df['is_profile_2'] = df['categories'].map(lambda x: 1 if x == "['Breakfast & Brunch', 'American (Traditional)', 'Restaurants']" else 0)
df['is_profile_3'] = df['categories'].map(lambda x: 1 if x == "['Wine Bars', 'Bars', 'American (New)', 'Nightlife', 'Restaurants']" else 0)
df['is_profile_4'] = df['categories'].map(lambda x: 1 if x == "['Fast Food', 'Restaurants']" else 0)
df['is_profile_5'] = df['categories'].map(lambda x: 1 if x == "['Fast Food', 'Mexican', 'Restaurants']" else 0)
df['is_profile_6'] = df['categories'].map(lambda x: 1 if x == "['American (Traditional)', 'Fast Food', 'Restaurants']" else 0)
df['is_profile_7'] = df['categories'].map(lambda x: 1 if x == "['Fast Food', 'Sandwiches', 'Restaurants']" else 0)
df['is_profile_8'] = df['categories'].map(lambda x: 1 if x == "['Burgers', 'Restaurants']" else 0)
df['is_profile_9'] = df['categories'].map(lambda x: 1 if x == "['Breakfast & Brunch', 'Restaurants']" else 0)
df['is_profile_10'] = df['categories'].map(lambda x: 1 if x == "['Delis', 'Fast Food', 'Sandwiches', 'Restaurants']" else 0)
df['is_profile_11'] = df['categories'].map(lambda x: 1 if x == "['Puerto Rican', 'Bars', 'Latin American', 'Caribbean', 'Nightlife', 'Restaurants']" else 0)
df['is_profile_12'] = df['categories'].map(lambda x: 1 if x == "['Irish', 'Music Venues', 'Arts & Entertainment', 'Bars', 'Pubs', 'Nightlife', 'Restaurants']" else 0)
df['is_profile_13'] = df['categories'].map(lambda x: 1 if x == "['Bars', 'Restaurants', 'American (Traditional)', 'Karaoke', 'Nightlife']" else 0)
df['is_profile_14'] = df['categories'].map(lambda x: 1 if x == "['Bars', 'American (Traditional)', 'Nightlife', 'Restaurants']" else 0)
df['is_profile_15'] = df['categories'].map(lambda x: 1 if x == "['Bars', 'Food', 'Beer, Wine & Spirits', 'Dive Bars', 'Nightlife', 'Karaoke']" else 0)
df['is_profile_16'] = df['categories'].map(lambda x: 1 if x == "['Bars', 'Restaurants', 'American (Traditional)', 'Sports Bars', 'Nightlife']" else 0)
df['is_profile_17'] = df['categories'].map(lambda x: 1 if x == "['Breakfast & Brunch', 'Cafes', 'American (Traditional)', 'Restaurants']" else 0)
df['is_profile_18'] = df['categories'].map(lambda x: 1 if x == "['Burgers', 'American (Traditional)', 'Restaurants']" else 0)
df['is_profile_19'] = df['categories'].map(lambda x: 1 if x == "['Fast Food', 'Mexican', 'Tex-Mex', 'Restaurants']" else 0)
df['is_profile_20'] = df['categories'].map(lambda x: 1 if x == "['Breakfast & Brunch', 'American (Traditional)', 'Diners', 'Restaurants']" else 0)

In [9]:
# Verify the columns and values were created as expected
df.is_profile_20.value_counts()

0    2449
1      51
Name: is_profile_20, dtype: int64

In [19]:
# Calculate the baseline accuracy. This is the probabiblity of predicting the city correctly, by always
# guessing the majority case i.e., not Las Vegas.

baseline_acc = 1. - df.is_lasvegas.mean()
baseline_acc

0.8564

In [20]:
# Q1: use KNN to predict whether business is in Las Vegas

from sklearn.neighbors import KNeighborsClassifier

predictor_matrix = df[["is_profile_1", "is_profile_2", "is_profile_3", "is_profile_4","is_profile_5","is_profile_6", "is_profile_7", "is_profile_8", "is_profile_9","is_profile_10","is_profile_11", "is_profile_12", "is_profile_13", "is_profile_14","is_profile_15","is_profile_16", "is_profile_17", "is_profile_18", "is_profile_19","is_profile_20"]]
y = df.is_lasvegas.values
X = predictor_matrix.iloc[:, 0:100]

knn = KNeighborsClassifier()

In [21]:
# Q2: predicting the business' location using KNN is marginally better
# than the baseline accuracy

from sklearn.cross_validation import cross_val_score

accs = cross_val_score(knn, X, y, cv=10)
print accs
print np.mean(accs)

[ 0.92430279  0.792       1.          1.          0.844       0.856       0.856
  0.74        0.856       0.85943775]
0.872774053985


In [13]:
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [14]:
knn.predict(X.iloc[0:10, :])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [295]:
knn.predict_proba(X.iloc[0:10, :])

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [15]:
predicted = knn.predict(X)

In [16]:
# Calculate the true and false positives/negatives:

tp = np.sum((y == 1) & (predicted == 1))
fp = np.sum((y == 0) & (predicted == 1))
tn = np.sum((y == 0) & (predicted == 0))
fn = np.sum((y == 1) & (predicted == 0))
print tp, fp, tn, fn

172 60 2081 187


In [17]:
from sklearn.metrics import confusion_matrix

In [18]:
# Confusion matrix same as above

confusion_matrix(y, predicted)

array([[2081,   60],
       [ 187,  172]])

In [ ]:
#3 : Scatter or pairplot could be used to visually represent model's results
#4: ?

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Different categories of ratings

---

Yelp is finally ready to admit that their rating system sucks. No one cares about the ratings, they just use the site to find out what's nearby. The ratings are simply too unreliable for people. 

Yelp hypothesizes that this is, in fact, because different people tend to give their ratings based on different things. They believe that perhaps some people always base their ratings on quality of food, others on service, and perhaps other categories as well. 

1. Do some users tend to talk about service more than others in reviews/tips? Divide up the tips/reviews into more "service-focused" ones and those less concerned with service.
2. Create two new ratings for businesses: ratings from just the service-focused reviews and ratings from the non-service reviews.
3. Construct a regression model for each of the two ratings. They should use the same predictor variables (of your choice). 
4. Validate the performance of the models.
5. Do the models coefficients differ at all? What does this tell you about the hypothesis that there are in fact two different kinds of ratings?

In [83]:
import pandas as pd
import seaborn as sns
import numpy as np

%matplotlib inline

project_id = "bigquery-dsi-andrew-stark"

sql = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.reviews] 
LIMIT 2500
"""

df_reviews = pd.read_gbq(sql, project_id=project_id)

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 2500 rows.

Total time taken 4.84 s.
Finished at 2016-12-20 10:17:42.


In [84]:
# Create a list of "service-focused" terms:
service_focused_terms = ['bottle_service','customer_service', 'food_service','friendly_staff', 'good_service', 'good_time', 'great_atmosphere','great_experience', 'great_place', 'great_service','great_time', 'place_good', 'place_great','really_enjoyed', 'really_good', 'really_like', 'really_nice','recommend_place', 'service_excellent', 'service_food','service_friendly', 'service_good', 'service_great','staff_friendly', 'super_friendly']

In [85]:
#Create a list of zeroes, to initially set the "is_service_focused" column
is_service_focused = []
for i in range(0,2500):
    is_service_focused.append(0)

In [86]:
# Set "is_service_focused" to 0 if there's a service-focused term used in the review

for item in service_focused_terms:
    presence = df_reviews[item].values
    for i in range(0,len(presence)):
        if presence[i]== 1:
            is_service_focused[i] = 1

In [87]:
#Creat a column in the dataframe, to track if there is a "service-focused term" in the review

df_reviews["is_service_focused"] = is_service_focused

In [88]:
df_reviews["is_service_focused"].value_counts()

0    1891
1     609
Name: is_service_focused, dtype: int64

In [82]:
# See the number of stars for each review

df_reviews.stars.value_counts()

3    963
1    635
2    560
4    342
Name: stars, dtype: int64

In [89]:
# Split the dataframe of reviews into 2 - service-focused and not  service-focused

df_is_service_focused = df_reviews[df_reviews["is_service_focused"]==1]
df_not_service_focused = df_reviews[df_reviews["is_service_focused"]!=1]

In [356]:
df_reviews.bottle_service.dtypes

dtype('int64')

In [303]:
import pandas as pd
import seaborn as sns
import numpy as np

%matplotlib inline

project_id = "bigquery-dsi-andrew-stark"

sql = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.tips] 
LIMIT 2500
"""

df_tips = pd.read_gbq(sql, project_id=project_id)

Requesting query... ok.
Query running...
Query done.
Processed: 84.0 Mb

Retrieving results...
Got 2500 rows.

Total time taken 5.76 s.
Finished at 2016-12-18 18:56:44.


In [304]:
df_tips.head()

,user_id,business_id,likes,date,hours_24,amazing_food,animal_style,awesome_food,awesome_place,awesome_service,...,service_good,service_great,slow_service,staff_friendly,staff_great,steak_eggs,super_friendly,sweet_potato,velvet_pancakes,worth_wait
0,VkTuaLUtRDIioAeicDGang,--jFTZmywe7StuZ2hEjxyA,0,2012-01-04,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3yLjzX-5AU76zIEkmUpiMQ,--jFTZmywe7StuZ2hEjxyA,0,2013-09-03,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,DDvHjP7wh5bqmIfqjcdwWw,--jFTZmywe7StuZ2hEjxyA,0,2013-05-18,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,fx1jNuoZjyn0t5g2KUeksw,-0HGqwlfw3I8nkJyMHxAsQ,0,2014-02-15,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,x0oIvyOtyH3rUDCfuUN40Q,-0HGqwlfw3I8nkJyMHxAsQ,0,2014-05-23,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Ran out of time. I'm spending lots of time trying to learn and understand the concepts, which leaves insufficient
# time to complete the project. Will review over Xmas break.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Identifying "elite" users

---

Yelp, though having their own formula for determining whether a user is elite or not, is interested in delving deeper into what differentiates an elite user from a normal user at a broader level.

Use a classification model to predict whether a user is elite or not. Note that users can be elite in some years and not in others.

1. What things predict well whether a user is elite or not?
- Validate the model.
- If you were to remove the "counts" metrics for users (reviews, votes, compliments), what distinguishes an elite user, if anything? Validate the model and compare it to the one with the count variables.
- Think of a way to visually represent your results in a compelling way.
- Give a brief write-up of your findings.


<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Find something interesting on your own

---

You want to impress your superiors at Yelp by doing some investigation into the data on your own. You want to do classification, but you're not sure on what.

1. Create a hypothesis or hypotheses about the data based on whatever you are interested in, as long as it is predicting a category of some kind (classification).
2. Explore the data visually (ideally related to this hypothesis).
3. Build one or more classification models to predict your target variable. **Your modeling should include gridsearching to find optimal model parameters.**
4. Evaluate the performance of your model. Explain why your model may have chosen those specific parameters during the gridsearch process.
5. Write up what the model tells you. Does it validate or invalidate your hypothesis? Write this up as if for a non-technical audience.

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. ROC and Precision-recall

---

Some categories have fewer overall businesses than others. Choose two categories of businesses to predict, one that makes your proportion of target classes as even as possible, and another that has very few businesses and thus makes the target varible imbalanced.

1. Create two classification models predicting these categories. Optimize the models and choose variables as you see fit.
- Make confusion matrices for your models. Describe the confusion matrices and explain what they tell you about your models' performance.
- Make ROC curves for both models. What do the ROC curves describe and what do they tell you about your model?
- Make Precision-Recall curves for the models. What do they describe? How do they compare to the ROC curves?
- Explain when Precision-Recall may be preferable to ROC. Is that the case in either of your models?